<a href="https://colab.research.google.com/github/abidd-23/machine_learning/blob/main/weather_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
!pip install tensorflow_probability==0.8.0rc0 --user --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2023.8.0 requires cloudpickle>=1.5.0, but you have cloudpickle 1.1.1 which is incompatible.
distributed 2023.8.0 requires cloudpickle>=1.5.0, but you have cloudpickle 1.1.1 which is incompatible.
dopamine-rl 4.0.6 requires tensorflow-probability>=0.13.0, but you have tensorflow-probability 0.8.0rc0 which is incompatible.
gym 0.25.2 requires cloudpickle>=1.2.0, but you have cloudpickle 1.1.1 which is incompatible.


In [3]:
import tensorflow_probability as tfp  # We are using a different module from tensorflow this time
import tensorflow as tf

In [ ]:
# We will model a simple weather system and try to predict the temperature on each day given the following information.

# 1-Cold days are encoded by a 0 and hot days are encoded by a 1.
# 2-The first day in our sequence has an 80% chance of being cold.
# 3-A cold day has a 30% chance of being followed by a hot day.
# 4-A hot day has a 20% chance of being followed by a cold day.
# 5-On each day the temperature is normally distributed with mean and standard deviation 0 and 5 on a cold day and mean and standard deviation 15 and 10 on a hot day.

In [7]:
tfd = tfp.distributions  # making a shortcut for later on
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])  # Refer to point 2 above
transition_distribution = tfd.Categorical(probs=[[0.3, 0.7],
                                                 [0.2, 0.8]])  # refer to points 3 and 4 above
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])  # refer to point 5 above

# the loc argument represents the mean and the scale is the standard devitation

In [8]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

In [9]:
mean = model.mean()

# due to the way TensorFlow works on a lower level we need to evaluate part of the graph
# from within a session to see the value of this tensor

# in the new version of tensorflow we need to use tf.compat.v1.Session() rather than just tf.Session()
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[ 3.       10.799999 11.579999 11.658    11.6658   11.66658  11.666658]
